In [7]:
import os.path  # 路径管理
import backtrader as bt
import pandas as pd

# 创建策略继承bt.Strategy
class TestStrategy(bt.Strategy):
    params = dict(
        period=1,  #均线参数
    )

    def log(self, txt, dt=None):
        # 记录策略的执行日志
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        '''
         line0=index4:close,line1=index3:low,line2=index2:high,line3=index1:open,line4=index5:vol,line6=index0:date
        self.datas[0] 在init函数中为最后一个k线序列数据,self.datas[1]为序列中第1个数据,self.datas[-1]为序列中前1个数据
        '''
        # 保存收盘价的引用
        self.dataclose = self.data0_close
        print('切片访问close前10个值', self.data0_close.array[:10])
        print('切片访问close后3个值', self.data0_close.array[-3:])
        print('切片访问close后3个值', self.data0_close.get(ago=-1, size=3))
        print('切片访问open后3个值', self.data0_open.get(ago=-1, size=3))
        print('close0', self.data0_close[0], 'close1', self.data0_close[1], 'close-1', self.data0_close[-1])
        # 加入均线指标
        self.sma = bt.indicators.SimpleMovingAverage(self.datas[0], period=self.params.period)



    def next(self):
        '''
        self.datas[0] 在next函数中为当下最新的数据,将从第一个开始遍历 self.datas[0]为前一个数据
        :return:
        '''
        # 记录收盘价
        # self.log('Close, %.2f' % self.dataclose[0])
        # price = self.datas[0]
        price = self.data0_open
        print(self.data0_datetime.date(0), price[0], self.sma[0],'price切片', price.get(ago=-1, size=2))
        # print('sma.get 切片访问前2个数据:', self.sma.get(ago=-1, size=2))
        pass


if __name__ == '__main__':
    # 创建Cerebro引擎
    cerebro = bt.Cerebro()
    # Cerebro引擎在后台创建broker(经纪人)，系统默认资金量为10000

    # 为Cerebro引擎添加策略
    cerebro.addstrategy(TestStrategy)

    # 获取当前运行脚本所在目录
    modpath = os.path.abspath('.')
    # 拼接加载路径
    datapath = os.path.join(modpath, '../../datas/orcl-1995-2014.txt')

    df = pd.read_csv(
        #Date,Open,High,Low,Close,Adj Close,Volume
        #1995-01-03,2.179012,2.191358,2.117284,2.117284,1.22,36301200
        filepath_or_buffer=datapath,
        sep=',',
        nrows=100,
        index_col=['Date'],  #设置行索引
        parse_dates=['Date'],  #解析时间
        date_parser=lambda x: pd.to_datetime(x, format='%Y-%m-%d'),  #时间解析的格式
        usecols=['Date', 'Open', 'High', 'Low', 'Adj Close', 'Volume'],
    )
    # 修改列名
    df = df.rename(columns={'Adj Close': 'Close'})
    # 截取时间段内样本数据
    df = df["1995-01-03":"1995-01-11"]
    # 使用pandas数据源创建交易数据集
    # 把它传递给backtrader数据源，然后添加到cerebro
    data = bt.feeds.PandasData(dataname=df)

    # 加载交易数据
    cerebro.adddata(data)

    # 设置投资金额100000.0
    cerebro.broker.setcash(100000.0)
    # 引擎运行前打印期出资金
    print('组合期初资金: %.2f' % cerebro.broker.getvalue())
    cerebro.run()
    # 引擎运行后打期末资金
    print('组合期末资金: %.2f' % cerebro.broker.getvalue())


组合期初资金: 100000.00
切片访问close前10个值 array('d', [1.22, 1.33, 1.44, 1.55, 1.66, 1.77, 1.88])
切片访问close后3个值 array('d', [1.66, 1.77, 1.88])
切片访问close后3个值 array('d', [1.55, 1.66, 1.77])
切片访问open后3个值 array('d', [2.092592, 2.135803, 2.191358])
close0 1.88 close1 1.22 close-1 1.77
1995-01-03 2.179012 1.22 price切片 array('d')
1995-01-04 2.123457 1.33 price切片 array('d')
1995-01-05 2.141975 1.44 price切片 array('d', [2.179012, 2.123457])
1995-01-06 2.092592 1.55 price切片 array('d', [2.123457, 2.141975])
1995-01-09 2.135803 1.66 price切片 array('d', [2.141975, 2.092592])
1995-01-10 2.191358 1.77 price切片 array('d', [2.092592, 2.135803])
1995-01-11 2.203704 1.88 price切片 array('d', [2.135803, 2.191358])
组合期末资金: 100000.00
